In [1]:
import doctest
import enum
import io
from collections import Counter
from dataclasses import dataclass
from functools import cached_property
from typing import List, Tuple

In [2]:
DATA = "input.txt"

# Part 1

Idea: hands are implemented as classes with comparison methods that checks the type attribute and, if necessary, the card strength(s). Cards can then be sorted to give the overall rank and then multiplied by the associated bids.

In [3]:
class Card(enum.IntEnum):
    C_2 = enum.auto()
    C_3 = enum.auto()
    C_4 = enum.auto()
    C_5 = enum.auto()
    C_6 = enum.auto()
    C_7 = enum.auto()
    C_8 = enum.auto()
    C_9 = enum.auto()
    C_T = enum.auto()
    C_J = enum.auto()
    C_Q = enum.auto()
    C_K = enum.auto()
    C_A = enum.auto()

    def __str__(self) -> str:
        return _card_to_str[self]

    @staticmethod
    def from_str(s: str) -> "Card":
        return _str_to_card[s] 

# should refactor to tidy this up! works for AoC 
_str_to_card = dict(zip("23456789TJQKA", list(Card)))
_card_to_str = {c: s for s, c in _str_to_card.items()}


class HandType(enum.IntEnum):
    HighCard    = enum.auto()
    OnePair     = enum.auto()
    TwoPair     = enum.auto()
    ThreeOfKind = enum.auto()
    FullHouse   = enum.auto()
    FourOfKind  = enum.auto()
    FiveOfKind  = enum.auto()


@dataclass
class Hand:
    cards: List[Card]

    def __str__(self) -> str:
        return ''.join([str(c) for c in self.cards])

    @staticmethod
    def from_str(s: str) -> "Hand":
        return Hand([Card.from_str(c) for c in s])

    def __lt__(self, other: "Hand") -> bool:
        """

        Example:

            >>> h1 = Hand.from_str("33332")
            >>> h2 = Hand.from_str("2AAAA")
            >>> h1 < h2
            False
        """
        if self.hand_type < other.hand_type:
            return True
        if self.hand_type > other.hand_type:
            return False
        for i in range(len(self.cards)):
            if self.cards[i] == other.cards[i]:
                continue
            return self.cards[i] < other.cards[i]

    @cached_property
    def hand_type(self) -> HandType:
        """Returns the ``HandType``.

        Example:

            >>> h = Hand([Card.C_A, Card.C_Q, Card.C_Q, Card.C_A, Card.C_2])
            >>> h.hand_type() == HandType.TwoPair
            True
        """
        cnts = Counter(self.cards)
        total, max_cnt = len(cnts), max(cnts.values())
        match total, max_cnt:
            case 1, _:
                return HandType.FiveOfKind
            case 2, 4:
                return HandType.FourOfKind
            case 2, _:
                return HandType.FullHouse
            case 3, 3:
                return HandType.ThreeOfKind
            case 3, 2:
                return HandType.TwoPair
            case 4, 2:
                return HandType.OnePair
            case 5, _:
                return HandType.HighCard
        raise ValueError(f"hand_type failed {self}")

In [4]:
def parse_input(data: io.TextIOBase, hand_cls: Hand = Hand) -> Tuple[List[Hand], List[int]]:
    """Returns a list of hands and a list of bids.

    Example:

        >>> data = io.StringIO('''32T3K 765
        ... T55J5 684
        ... KK677 28
        ... KTJJT 220
        ... QQQJA 483''')
        >>> hands, bids = parse_input(data)
        >>> for h, b in zip(hands, bids):
        ...     print(b, h)
        765 32T3K
        684 T55J5
        28 KK677
        220 KTJJT
        483 QQQJA
    """
    hands = []
    bids = []
    for line in data:
        hand_str, bid_str = line.split()
        hands.append(hand_cls.from_str(hand_str))
        bids.append(int(bid_str))
    return hands, bids

In [5]:
def total_winnings(hands: List[int], bids: List[int]) -> int:
    """Returns the total winnings.

    Example:

        >>> data = io.StringIO('''32T3K 765
        ... T55J5 684
        ... KK677 28
        ... KTJJT 220
        ... QQQJA 483''')
        >>> hands, bids = parse_input(data)
        >>> total_winnings(hands, bids)
        6440
    """
    hand_bids = list(zip(hands, bids))
    hand_bids = sorted(hand_bids, key=lambda hb: hb[0])
    return sum(rank*bid for rank, (_, bid) in enumerate(hand_bids, start=1))

In [6]:
doctest.testmod()

TestResults(failed=0, attempted=9)

In [7]:
with open(DATA, "r") as f:
    hands, bids = parse_input(f)
total_winnings(hands, bids)

250946742

# Part 2

The `hand_type` logic needs updating to account for the joker being a wildcard and the `__lt__` logic needs updating to account for the joker now being the weakest card. 

The `hand_type` logic reassigns the joker counts to the card with highest count.

Ideally the `__lt__` logic would "just work" as a `JokerCard` subclass or something could be created with a lower joker value but Python doesn't allow enum subclasses so rewriting `__lt__` seems simplest, albiet not ideal, for an AoC implementation.

In [8]:
class JokerHand(Hand):
    @staticmethod
    def from_str(s: str) -> "Hand":
        return JokerHand([Card.from_str(c) for c in s])
    
    def __lt__(self, other: "JokerHand") -> bool:
        """

        Example:

            >>> h1 = JokerHand.from_str("Q3QQQ")
            >>> h2 = JokerHand.from_str("QJQQ4")
            >>> h1 < h2
            False
        """
        if self.hand_type < other.hand_type:
            return True
        if self.hand_type > other.hand_type:
            return False
        for i in range(len(self.cards)):
            if self.cards[i] == other.cards[i]:
                continue
            c_i = int(self.cards[i])
            o_i = int(other.cards[i])
            if c_i == int(Card.C_J):
                c_i = Card.C_2 - 1
            if o_i == int(Card.C_J):
                o_i = Card.C_2 - 1
            return c_i < o_i

    @cached_property
    def hand_type(self) -> HandType:
        cnts = Counter(self.cards)
        if Card.C_J in cnts and len(cnts) > 1:
            cnt_j = cnts.pop(Card.C_J)
            max_card = max(cnts.items(), key=lambda x: x[1])[0]
            cnts[max_card] += cnt_j
        total, max_cnt = len(cnts), max(cnts.values())
        match total, max_cnt:
            case 1, _:
                return HandType.FiveOfKind
            case 2, 4:
                return HandType.FourOfKind
            case 2, _:
                return HandType.FullHouse
            case 3, 3:
                return HandType.ThreeOfKind
            case 3, 2:
                return HandType.TwoPair
            case 4, 2:
                return HandType.OnePair
            case 5, _:
                return HandType.HighCard
        raise ValueError(f"hand_type failed {self}")

In [9]:
doctest.testmod()

TestResults(failed=0, attempted=12)

In [10]:
with open(DATA, "r") as f:
    hands, bids = parse_input(f, hand_cls=JokerHand)
total_winnings(hands, bids)

251824095